# TP SQL Agrégation Informatique

date du document: 23 septembre 2024

Le système utilisé pendant les TME est H2.

- Connectez-vous à l'environnement Jupyterhub de Capsule  (section TMEs Jupyter Notebook)
- Importez les fichiers téléchargés dans votre environnement  (flèche vers le haut en haut à gauche)
- Sélectionnez un Kernel 


Alternatives si problème d'utilisation de Jupyterhub de Capsule:

- Installer et exécuter le serveur H2 en local (voir le documentation la documentation SGBD H2)
- Télécharger les fichiers du TME dans un répertoire local et lancer jupyter-lab ou jupyter-notebook en local (installer  java si besoin), il n'y a pas de kernel pyspark dans ce cas


# Installation H2 

Le système utilisé pendant les TME est H2 qui est accédée avec l'adaptateur PostgreSQL:

In [ ]:
pip install psycopg2-binary

**Relancez le kernel**: Kernel -> Restart kernel ...

Télécharger les pilote PostgreSQL et H2 (optionnel)

In [2]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar -O postgresql-42.6.0.jar
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar -O h2-2.1.214.jar

--2024-10-07 18:19:15--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/postgresql-42.6.0.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1081604 (1,0M) [application/octet-stream]
Enregistre : ‘postgresql-42.6.0.jar’

postgresql-42.6.0.j 100%[===================>]   1,03M  --.-KB/s    ds 0,07s   

2024-10-07 18:19:15 (13,8 MB/s) - ‘postgresql-42.6.0.jar’ enregistré [1081604/1081604]

--2024-10-07 18:19:15--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/h2-2.1.214.jar
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 2543011 (2,4M) [application/octet-stream]
Enregistre : ‘h2-2.1.214.jar’

h2-2.1.214.jar      100%[===================>]   2,42M  10,1MB/s    ds 0,

# Démarrer le serveur SQL H2

In [1]:
import os
import subprocess
import psycopg2

def connect_H2(db,user,password,port=5435):
    global connection
    try:
        connection
    except:
        connection = None
    if connection != None:
        try:
            connection.close()
            print("Connection closed")
        except  Error as e:
            print(f"The error '{e}' occurred")
    try:
        # connection = sqlite3.connect(path,isolation_level='DEFERRED')
        connection = psycopg2.connect(f"dbname={db} user={user} password={password} host=localhost port={port}")
        print("Connection to H2 DB successful")
    except Exception as e:
        print(f"The error '{e}' occurred")
    return connection

def execute(connection, query, show=True,close=True):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        print(cursor.rowcount,"rows")
        if show and cursor.rowcount and cursor.description is not None:
            names = [desc[0] for desc in cursor.description]
            lengths={}
            for attr in names:
                lengths[attr]=len(attr)
            for ligne in cursor:
                i=0
                for attr in ligne:
                    lengths[names[i]]=max(lengths[names[i]],len(str(attr).replace('\n',' ')))
                    i=i+1
            print('|',end='')
            for attr in names:
                print(str(attr).ljust(lengths[attr]),end='|')
            print()
            print('|',end='')
            for attr in names:
                print(''.ljust(lengths[attr]+1,'-'),end='')
            print()
            cursor.execute(query)
            for ligne in cursor:
                i=0
                print('|',end='')
                for attr in ligne:
                    print(str(attr).replace('\n',' ')[:lengths[names[i]]].ljust(lengths[names[i]]),end='|')
                    i=i+1
                print()
        if close:
            cursor.close()
    except Exception as e:
        print(f"The error '{e}' occurred")
        cursor=None
    return cursor


def show_table(connection,table_name):

    query=f"""
SELECT table_name,
       column_name,
       data_type,
       column_default,
       is_nullable,
       character_maximum_length,
       numeric_precision
FROM information_schema.columns 
where lower(table_name)  = '{table_name}'  
"""
    execute(connection,query)

def show_schema(connection):

    print('*********** Tables ************')
    query="""
    select TABLE_NAME
    from INFORMATION_SCHEMA.TABLES 
    where TABLE_SCHEMA = 'public'
    """
    execute(connection,query,show=True)

    print('\n\n*********** Domaines ************')
    query="""
    SELECT domain_name,check_clause 
    FROM information_schema.domain_constraints a, information_schema.check_constraints b
    where a.constraint_name=b.constraint_name
    """
    execute(connection,query)

    print('\n\n*********** Attributs ************')
    query=f"""
    SELECT c.table_name,
           c.column_name,
           c.data_type,
           c.column_default,
           c.is_nullable,
           c.character_maximum_length,
           c.numeric_precision
    FROM INFORMATION_SCHEMA.TABLES t, information_schema.columns c
    where t.table_name=c.table_name
    and t.TABLE_SCHEMA = 'public'
    """
    execute(connection,query)
    

local_dir = "./data"
os.makedirs(local_dir, exist_ok=True)

# binaire java : décommenter sur jupyterhub Moodle
# java='/opt/tljh/user/envs/pyspark/bin/java'
java='java'

# chaque utilisateur doit avoir un port différent!
port = 5010

print(f'La commande java est: {java}')
print(f'Le numero du port du serveur H2 est: {port}')
print(f'Les fichiers dans {local_dir}:')
for f in os.listdir(local_dir):
    print(f'   {f}')

La commande java est: java
Le numero du port du serveur H2 est: 5010
Les fichiers dans ./data:
   arbres5010.mv.db
   employes5010.mv.db
   jo5008.mv.db
   jo5010.mv.db
   villes5010.mv.db


In [2]:
# Défnir le port pour le serveur H2
cmd=f"{java} -Dh2.bindAddress=127.0.0.1  -cp  h2-2.1.214.jar:postgresql-42.6.0.jar org.h2.tools.Server -pg -pgPort {port}  -baseDir ./data -ifNotExists"
proc=subprocess.Popen(cmd,
                      shell=True,
                      text=True,
                      stdout=subprocess.PIPE,
                      stderr=subprocess.PIPE, 
                      )
try:
    outs, errs = proc.communicate(timeout=1)
    print(errs)
except :
    print("Serveur H2 en marche")

Exception in thread "main" org.h2.jdbc.JdbcSQLNonTransientConnectionException: Exception à l'ouverture du port "5010" (le port est peut-être en cours d'utilisation), cause: "java.net.BindException: Adresse déjà utilisée (Bind failed)"
Exception opening port "5010" (port may be in use), cause: "java.net.BindException: Adresse déjà utilisée (Bind failed)" [90061-214]
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:678)
	at org.h2.message.DbException.getJdbcSQLException(DbException.java:477)
	at org.h2.message.DbException.get(DbException.java:212)
	at org.h2.util.NetUtils.createServerSocketTry(NetUtils.java:211)
	at org.h2.util.NetUtils.createServerSocket(NetUtils.java:177)
	at org.h2.server.pg.PgServer.start(PgServer.java:174)
	at org.h2.tools.Server.start(Server.java:521)
	at org.h2.tools.Server.runTool(Server.java:331)
	at org.h2.tools.Server.main(Server.java:126)
Caused by: java.net.BindException: Adresse déjà utilisée (Bind failed)
	at java.base/java.net.PlainSock

# TD et TME4 : PREMIERS PAS EN SQL

Ce TD/TME utilise les données contenues dans les fichiers **bd_jo_v1_H2.sql** et **bd_jo_v2_H2.sql**


## TD4 

Ce TD utilise les données contenues dans les fichiers **bd_jo_v1_H2.sql** 

BASE DE DONNÉES **JEUX OLYMPIQUES D'HIVER 2014** <br/>
On considère le schéma de la base JEUXOLYMPIQUE2014 qui décrit les athlètes et leurs résultats aux
épreuves des Jeux Olympiques d'Hiver Sotchi 2014 :<br/>


**ATHLETE** ( <u>NOM,PRENOM</u>,DATENAISSANCE,PAYS)<br/>
**RANGEPREUVE**( <u>NOMATH*,PRENOMATH*,EPREUVE,SPORT,CATEGORIE</u>,DATEDEBUT,DATEFIN,RANG)<br/>

La relation ATHLETE contient des informations concernant les athlètes ayant participé aux Jeux
Olympiques. Plus précisément : leur nom, leur prénom, leur date de naissance si elle est renseignée
(ou NULL sinon) ainsi que le pays qu'ils représentent.
La relation RANGEPREUVE renvoie pour chaque athlète identifie par NOMATH ET PRENOMATH et
chaque épreuve identifiée par SPORT, EPREUVE ET CATEGORIE (HOMMES OU FEMMES) à laquelle il a
participé, le RANG qu'il a obtenu. On pour chaque épreuve les dates de début et de fin (DATEDEBUT,
DATEFIN).
Lorsqu'un athlète participe à une épreuve puis est disqualifié, son rang n'est pas renseigné (NULL)



### Télécharger les données, créer et charger les données

Télécharger le fichier SQL **bd_jo_v1_H2.sql** contenant les données de la base JO.

In [3]:
!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/bd-jo-v1_H2.sql -O bd-jo-v1_H2.sql

--2024-10-07 18:24:05--  https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/bd-jo-v1_H2.sql
Résolution de nuage.lip6.fr (nuage.lip6.fr)… 132.227.201.11
Connexion à nuage.lip6.fr (nuage.lip6.fr)|132.227.201.11|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 455744 (445K) [application/octet-stream]
Enregistre : ‘bd-jo-v1_H2.sql’

bd-jo-v1_H2.sql     100%[===================>] 445,06K  --.-KB/s    ds 0,04s   

2024-10-07 18:24:05 (9,74 MB/s) - ‘bd-jo-v1_H2.sql’ enregistré [455744/455744]



### Connexion à H2

In [4]:
connection = connect_H2(f"jo{port}","ba","ba",port=port)

Connection to H2 DB successful


In [5]:
execute(connection,"drop all objects")

0 rows


<cursor object at 0x7ef849b40f40; closed: -1>

### Charger les données bd_jo_v1_H2.sql

In [6]:
schemafile=open("bd-jo-v1_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection, create_schema)
connection.commit()

0 rows


### Requêtes

**Q1**. Les athlètes français.<br/>
Résultat : (92 lignes)

In [9]:
query="""
select * from Athlete where pays='France'
"""

In [ ]:
# Afficher le nombre de n-uplets (par defaut)
execute(connection,query, show=False)

In [ ]:
# Afficher le résultat (par defaut)
execute(connection,query)

**Q2**. La date de naissance de Denis TEN.<br/>
Résultat: 1993-06-13

In [ ]:
query="""
select dateNaissance
from Athlete
where nom='TEN' and prenom='Denis'
"""
execute(connection,query)

**Q3**. Les épreuves (sport, épreuve) dans lesquelles Therese JOHAUG a gagné une médaille, ainsi
que le rang qu'elle a obtenu.<br/>
Indice : pour gagner une médaille, il faut avoir été classé dans le rang 1, 2 ou 3.<br/>
Résultat : ('Ski de fond', '10km', 3) et ('Ski de fond', '30km', 2)

In [ ]:
query="""
select sport, epreuve, rang
from RangEpreuve
where nomAth='JOHAUG' and prenomAth='Therese' and rang <= 3
"""
execute(connection,query)

**Q4**. La nationalité des athlètes Kamil STOCH et Suk-Hee SHIM.<br/>
Résultat : ('SHIM', 'Suk-Hee', 'Corée du Sud') et ('STOCH', 'Kamil', 'Pologne')

In [ ]:
query="""
select nom, prenom, pays
from Athlete
where (nom='STOCH' and prenom='Kamil') or (nom='SHIM' and prenom='Suk-Hee')
"""
execute(connection,query)

In [ ]:
query="""
select nom, prenom, pays from Athlete where (nom='STOCH' and prenom='Kamil')
union
select nom, prenom, pays from Athlete where (nom='SHIM' and prenom='Suk-Hee')"""
execute(connection,query)

**Q5**. Tous les sports des Jeux Olympiques.<br/>
Résultat : (12 lignes)

In [ ]:
query="""
select distinct sport from RangEpreuve
"""
execute(connection,query)

**Q6**. Le nom des athlètes originaire de Scandinavie (pays : Danemark, Finlande, Norvège, Suède,
Islande) triés par le nom, puis par leur prénom.<br/>
Résultat : (186 lignes)

In [ ]:
query="""
select distinct nom , prenom
from Athlete
where pays in ('Danemark', 'Finlande', 'Norvège', 'Suède', 'Island')
order by nom, prenom
"""
execute(connection,query)

In [ ]:
query="""
select distinct nom , prenom
from Athlete
Where pays = 'Danemark' or pays= 'Finlande' or pays= 'Norvège'
or pays='Suède' or pays= 'Island'
order by nom, prenom
"""
execute(connection,query)

**Q7**. Le nom de sports contenant la chaîne de caractères 'ski' (par exemple, 'Ski de fond', 'Saut à
ski', …).<br/>
Résultat : Ski de fond, Ski alpin, Ski acrobatique, Saut à ski

In [ ]:
query="""
select distinct sport
from RangEpreuve
where lower(sport) like '%ski%'
"""
execute(connection,query)

**Q8**. Les athlètes (nom, prénom) dont la date de naissance n'est pas renseignée.<br/>
Résultat : (1516 lignes)

In [ ]:
query="""
select * from Athlete where dateNaissance is null
"""
execute(connection,query)

**Q9**. Les athlètes qui ont participé à l'épreuve '10km' de 'Ski de fond' sans être disqualifié (rang
inconnu), triés par rang.<br/>
Résultat : (75 lignes) 

In [ ]:
query="""
select nomAth, prenomAth, rang
from RangEpreuve
where sport='Ski de fond' and epreuve='10km'
and rang is not null
order by rang
"""
execute(connection,query)

**Q10**. Les épreuves (sport, épreuve et categorie) qui ont eu lieu après le 21 février 2014.<br/>
Résultat : (4 lignes)

In [ ]:
query="""
select distinct SPORT,EPREUVE ,CATEGORIE
from RANGEPREUVE where DATEDEBUT > '2014-02-21'
"""
execute(connection,query)

In [ ]:
query="""
select distinct SPORT ,EPREUVE ,CATEGORIE
from RANGEPREUVE
where DATEDEBUT > parsedatetime('2014-02-21','yyyy-MM-dd')
"""
execute(connection,query)

In [ ]:
query="""
select distinct SPORT ,EPREUVE ,CATEGORIE
from RANGEPREUVE
where DATEDEBUT > parsedatetime('21 févr. 2014','dd MMM yyyy')"""
execute(connection,query)

In [ ]:
query="""
Select distinct SPORT ,EPREUVE ,CATEGORIE
from RANGEPREUVE
where DATEDEBUT > parsedatetime('21 février 2014','dd MMMM yyyy')
"""
execute(connection,query)

**Q11**. Les épreuves (sport, épreuve et categorie) qui ont duré entre 2 et 4 jours, ainsi que leur
durée exacte.<br/>
Résultat : (16 lignes)

In [ ]:
query="""
select distinct sport, epreuve, categorie, datediff(DAY, dateDebut, dateFin)+1,
datedebut, datefin
from RangEpreuve
where datediff(DAY, dateDebut, dateFin)+1 >= 2
and datediff(DAY, dateDebut, dateFin)+1 <=4
"""
execute(connection,query)

In [ ]:
query="""
select distinct sport, epreuve, categorie, DATEDIFF(DAY, dateDebut, dateFin)+1
from RangEpreuve
where datediff(DAY, dateDebut, dateFin)+1 between 2 and 4
"""
execute(connection,query)

**Q12**. Les épreuves (sport, épreuve et catégorie) dont le final a eu lieu pendant un weekend.<br/>
Résultat : (26 lignes)

In [ ]:
query="""
select distinct sport, epreuve, categorie, dayname(dateFin)
from RangEpreuve
where dayname(dateFin) in ('Saturday', 'Sunday')
"""
execute(connection,query)

In [ ]:
query="""
select distinct sport, epreuve, categorie, dayname(dateFin), dayofweek(dateFin)
from RangEpreuve
where dayofweek(dateFin) in (6,7)
"""
execute(connection,query)

In [ ]:
query="""
select distinct sport, epreuve, categorie, dayname(dateDebut), dayname(dateFin)
from RangEpreuve
where dayofweek(dateFin) = 6 or dayofweek(dateFin)=7
"""
execute(connection,query)

**Q13**. Les athlètes (nom, prénom) femmes de nationalité française.<br/>
Résultat : (26 lignes)

In [ ]:
query="""
select a.nom, a.prenom from athlete a where pays = 'France'
intersect
select r.nomath, r.prenomath from rangepreuve r where r.categorie = 'Femmes'
"""
execute(connection,query)

In [ ]:
query="""
select a.nom, a.prenom from athlete a where pays = 'France'
intersect
select r.nomath as nom, r.prenomath as prenom from rangepreuve r
where r.categorie = 'Femmes'
"""
execute(connection,query)

**Q14**. Les athlètes (nom, prénom) qui ont aujourd'hui moins de 30 ans et qui ont gagné une
médaille.<br/>
Résultat : depend de la date d'exécution 

In [ ]:
query="""
select a.nom, a.prenom from athlete a
where DATEDIFF(YEAR,datenaissance,now()) <= 30
intersect
select r.nomath, r.prenomath from rangepreuve r
where r.rang between 1 and 3
"""
execute(connection,query)

In [ ]:
query="""
select distinct a.NOM ,a.PRENOM
from ATHLETE a, RANGEPREUVE r
where extract(year from current_date)-extract(year from a.DATENAISSANCE) <= 30
and r.rang<=3 and a.nom=r.nomAth and a.prenom=r.prenomAth
"""
execute(connection,query)

In [ ]:
query="""
Select a.nom, a.prenom from athlete a
where datediff(YEAR,datenaissance,now()) < 30
    or (datediff(YEAR,datenaissance,now()) = 30
        and day_of_year(datenaissance) > day_of_year(now()))
intersect
select r.nomath, r.prenomath
from rangepreuve r
where r.rang between 1 and 3
"""
execute(connection,query)

**Q15**. Les athlètes (nom, prénom) qui avaient leur anniversaire à la date de fin de l'épreuve dans
laquelle ils participaient.<br/>
Résultat : ('GARANICHEV', 'Evgeniy', 13, 2), ('SKOBREV', 'Ivan', 8, 2) et ('WEIBRECHT',
'Andrew', 10, 2)

In [ ]:
query="""
select nom, prenom, extract(day from dateNaissance), extract(month from dateNaissance)
from Athlete
intersect
select nomAth, prenomAth, extract(day from dateFin), extract(month from dateFin)
from RangEpreuve
"""
execute(connection,query)

In [ ]:
query="""
select NOMATH ,PRENOMATH, day(DATEFIN), month(datefin)
from RANGEPREUVE
intersect
select NOM ,PRENOM, day(DATENAISSANCE), month(datenaissance)
from ATHLETE
"""
execute(connection,query)

**Q16**.  Les athlètes français (nom, prénom) qui ont gagné une médaille.<br/>
Résultat : (12 lignes)

In [ ]:
query="""
select nom, prenom from Athlete where pays='France'
intersect
select nomAth, prenomAth from RangEpreuve where rang <= 3
"""
execute(connection,query)

**Q17**. Les sports qui n'ont pas d'épreuves féminines.<br/>
Résultat : Combiné nordique

In [ ]:
query="""
select distinct sport from RangEpreuve
except
select sport from RangEpreuve where categorie='Femmes'
"""
execute(connection,query)

## TME4 

Ce TME utilise les données contenues dans les fichiers **bd_jo_v1_H2.sql** et **bd_jo_v2_H2.sql**


### Exemple d'utilisation H2

Tous les athlètes

In [ ]:
cursor=execute(connection,query="SELECT * FROM athlete",show=True)

In [ ]:
cursor.query

In [ ]:
cursor.rowcount

In [ ]:
cursor.description

Le nombre de n-uplets stockés dans la relation Athlete.

In [ ]:
cursor=execute(connection,query="select count(*) from Athlete", show=True)

### Exercice 1

Exprimer les requêtes suivantes en SQL 

**Q1**. Les athlètes (nom, prénom) d'Inde (104 lignes).<br/>
Résultat : IQBAL Nadeem, KESHAVAN Shiva, THAKUR Himanshu

In [ ]:
query="""
select *
from Athlete
where pays='Inde'
"""
execute(connection,query)

**Q2** Le prénom des athlètes dont le nom est 'GOLD'.<br/>
Résultat : Arielle, Gracie, Taylor

In [ ]:
query="""
select prenom
from Athlete
where nom='GOLD'
"""
execute(connection,query)

**Q3** La nationalité de AONO Ryo.<br/>
Résultat : Japon

In [ ]:
query="""
select pays
from Athlete
where nom='AONO' and prenom='Ryo'
"""
execute(connection,query)

**Q4** Le gagnant du médaille d'or de chaque épreuve. Triez le résultat par sport, puis par épreuve.<br/>
Résultat : (74 lignes)

In [ ]:
query="""
select sport, epreuve, nomAth, prenomAth
from RangEpreuve
where rang = 1
order by sport, epreuve
"""
execute(connection,query)

**Q5** Toutes les épreuves (sport, épreuve, catégorie) triées par la date de fin de l'épreuve.   
Résultat : (73 lignes)

In [ ]:
query="""
select distinct sport, epreuve, categorie, dateFin
from RangEpreuve
order by dateFin
"""
execute(connection,query)

**Q6** Les athlètes nés entre 1990 et 1995.<br/>
Résultat : (87 lignes)

In [ ]:
query="""
select nom, prenom, dateNaissance
from Athlete
where extract (year from dateNaissance) between 1990 and 1995
"""
execute(connection,query)

**Q7** Les athlètes suisses ayant participé au sport 'Biathlon' et disqualifié à au moins une épreuve
dece sport (solution sans jointure, avec une opération ensembliste).<br/>
Résultat : GASPARIN Elisa

In [ ]:
query="""
(select nom, prenom
from athlete
where pays='Suisse')
intersect
(select nomAth, prenomAth
from RangEpreuve
where sport='Biathlon' and rang is null)
"""
execute(connection,query)

**Q8** Les épreuves dans lesquelles il n'y a pas eu une médaille d'argent (solution sans jointure,
avec une opération ensembliste). <br/>
Résultat : Ski alpin Descente Femmes

In [ ]:
query="""
(select distinct sport, epreuve, categorie from RangEpreuve)
except
(select distinct sport, epreuve, categorie from RangEpreuve where rang=2)
"""
execute(connection,query)

### Exercice 2

On considère maintenant le schéma complet de la base JO2014

**PAYS** (<u>CODEPAYS</u>, NOMP)<br/>
**SPORT** (<u>SID</u>, NOMSP)<br/>
**EPREUVE** (<u>EPID</u>, SID*, NOMEP, CATÉGORIE, DATEDEBUT, DATEFIN) <br/>
**ATHLETE** (<u>AID</u>, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*) EQUIPE (EQID, CODEPAYS*)<br/>
**ATHLETESEQUIPE** (<u>EQID*, AID*</u>)<br/>
**RANGINDIVIDUEL** (<u>EPID*, AID*</u>, RANG)<br/>
**RANGEQUIPE** (<u>EPID*, EQID*</u>, RANG)<br/>
    

La relation PAYS contient le code et le nom de tous les pays, même si ils n'ont pas participé aux Jeux Olympiques. Les sports (n-uplets de la relation SPORT) sont un ensemble d'épreuves (n-uplets de la relation EPREUVE). Pour chaque épreuve on connaît son nom et les date de début et fin de l'épreuve. Les épreuves peuvent être individuelles ou par équipe. Dans le premier cas, la participation des athlètes (n-uplets de la relation ATHLETE) est stocké dans la table RANGINDIVIDUEL qui contient en plus le rang qu'ils ont obtenu (1 pour la médaille d'or). Pour les épreuves par équipe les résultats sont stockés dans la relation RANGEQUIPE, alors que l'information sur le pays de chaque équipe et ses participants et stocké dans les relations EQUIPE et ATHLETESEQUIPE. Dans les relations RANGINDIVIDUEL et RANGEQUIPE l'attribut rang est égal à null si l'athlète ou l'équipe a été disqualifié.


#### Télécharger les données, créer et charger les données

Télécharger le fichier SQL **bd_jo_v2_H2.sql** contenant les données de la base JO.

In [ ]:
#PAS TOUCHE
#!wget https://nuage.lip6.fr/s/LqD9N23kxrfHopr/download?path=agreg/bd-jo-v2_H2.sql -O bd-jo-v2_H2.sql

Créer les tables et charger les données bd_jo_v2_H2.sql

In [ ]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection, create_schema)
connection.commit()

#### Requêtes

Exprimer les requêtes suivantes en SQL :

**Q9**. Les épreuves (sport, épreuve, catégorie) individuelles.<br/>
Résultat : (73 lignes)

In [ ]:
query="""
select distinct s.nomSp, e.nomEp, e.categorie
from Epreuve e, Sport s, RangIndividuel r
where e.sid=s.sid and r.epid=e.epid
"""
execute(connection,query)


**Q10**. Les résultats (nom, prénom, pays de l'athlète et rang) de l'épreuve '1000m' du 'Patinage de vitesse' pour les 'Femmes'.<br/>
Résultat : (36 lignes)

In [ ]:
query="""
select a.nomAth, a.prenomAth, a.codePays, r.rang
from Sport s, Epreuve e, RangIndividuel r, Athlete a
where s.nomSp='Patinage de vitesse' and e.nomEp='1000m' and e.categorie='Femmes'
and e.sid=s.sid and r.epid=e.epid and r.aid=a.aid
"""
execute(connection,query)

**Q11**. Le nom et prénom des athlètes qui forment l'équipe qui a gagné la médaille d'or dans l'épreuve 'relais 4x6km' de 'Biathlon' de 'Femmes'.   
Résultat : SEMERENKO Vita, SEMERENKO Valj, DZHYMA Juliya, PIDHRUSHNA Olena

In [ ]:
query="""
select a.nomAth, a.prenomAth
from Athlete a, AthletesEquipe ae, Epreuve e, Sport s, RangEquipe r
where e.nomEp='relais 4x6km' and e.categorie='Femmes'
and e.sid=s.sid and s.nomSp='Biathlon'
and r.epid=e.epid and r.rang=1 and r.eqid=ae.eqid
and a.aid=ae.aid
"""
execute(connection,query)

# TD et TME 5

### Charger bd-jo-v2_H2.sql

In [ ]:
schemafile=open("bd-jo-v2_H2.sql",mode="r",encoding='utf-8')
create_schema=schemafile.read()
execute(connection, create_schema)
connection.commit()

### Jointures

Q1. Les athlètes français (nom pays = 'France') (104 lignes).

In [ ]:
query="""
select a.nomAth, a.prenomAth
from Pays p, Athlete a
where a.codePays = p.codePays and p.nomP='France'
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select nomAth, prenomAth
from Pays natural join Athlete 
where nomP='France'
"""
cursor=execute(connection,query,show=True)

Q2. Les épreuves (sport, nom d'épreuve, catégorie) triées par nom de sport, puis par nom d'épreuve dans l'ordre inverse du dictionnaire (98 lignes).

In [ ]:
query="""
select distinct e.nomep, e.categorie, s.nomsp
from epreuve e, sport s
where e.sid = s.sid
order by s.nomsp desc, e.nomep desc
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select nomep, categorie, nomsp
from epreuve natural join sport 
order by nomsp desc, nomep desc
"""

cursor=execute(connection,query,show=True)

Q3. Les athlètes ayant participé à une équipe de France (30 lignes).

In [ ]:
query="""
select distinct a.nomAth, a.prenomAth
from Athlete a, Equipe e, AthletesEquipe ae, Pays p
where a.aid=ae.aid and e.eqid=ae.eqid
and e.codePays=p.codePays and p.nomP='France'
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct nomAth, prenomAth
from ((Athlete natural join Equipe) natural join AthletesEquipe)
     natural join Pays 
where nomP='France'
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct nomAth, prenomAth
from Pays natural join Athlete natural join AthletesEquipe natural join Equipe
where nomP='France'
"""

cursor=execute(connection,query,show=True)

In [ ]:
# reponse fausse !!!
query="""
select distinct nomAth, prenomAth
from Athlete natural join Equipe natural join AthletesEquipe
     natural join Pays 
where nomP='France'
"""

cursor=execute(connection,query,show=True)

Q4. Les épreuves (sport, épreuve, catégorie) auxquelles participent des équipes (25 lignes).

In [ ]:
query="""
select distinct s.nomSp, e.nomEp, e.categorie
from Epreuve e, Sport s, RangEquipe r
where e.sid=s.sid and r.epid=e.epid
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct nomSp, nomEp, categorie
from (Epreuve natural join RangEquipe) natural join Sport 
"""

cursor=execute(connection,query,show=True)

Q5. Le pays qui a gagné, en équipe, la médaille d'or dans l'épreuve de la catégorie 'Femmes' intitulée 'relais 4x6km' du sport 'Biathlon' (Ukraine).

In [ ]:
query="""
select p.nomP
from Epreuve ep, Sport s, RangEquipe r, Equipe eq, Pays p
where ep.nomEp='relais 4x6km' and ep.categorie='Femmes'
and ep.sid=s.sid and s.nomSp='Biathlon'
and r.epid=ep.epid and r.rang=1 and r.eqid=eq.eqid
and eq.codePays=p.codePays
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct nomp
from Sport
natural join Epreuve 
natural join RangEquipe
natural join Equipe 
natural join Pays 
where nomEp='relais 4x6km' and categorie='Femmes'
and nomSp='Biathlon' and rang=1 
"""

cursor=execute(connection,query,show=True)

In [ ]:
# ne marche pas 
query="""
select distinct nomp
from Pays natural join Epreuve natural join RangEquipe natural join Equipe natural join Sport 
where nomEp='relais 4x6km' and categorie='Femmes'
and nomSp='Biathlon' and rang=1 
"""

cursor=execute(connection,query,show=True)

Q6. Les athlètes ayant participé à au moins une épreuve individuelle et au moins une par équipe (372 lignes).

In [ ]:
query="""
select distinct a.*
from Athlete a, RangIndividuel ri, RangEquipe re, AthletesEquipe ae
where a.aid=ri.aid and re.eqid=ae.eqid and a.aid=ae.aid
"""

cursor=execute(connection,query,show=True)

Q7. Les homonymes (les nom de familles portés par deux athlètes ou plus) (141 lignes)

In [ ]:
query="""
select distinct a1.nomAth
from Athlete a1, Athlete a2
where a1.aid <> a2.aid and a1.nomAth = a2.nomAth
"""

cursor=execute(connection,query,show=True)

Q8. Les athlètes ayant participé à (au moins) deux épreuves individuelles (706 lignes).

In [ ]:
query="""
select distinct a.*
from Athlete a, RangIndividuel r1, RangIndividuel r2
where a.aid = r1.aid and a.aid = r2.aid and r1.epid <> r2.epid
"""

cursor=execute(connection,query,show=True)

Q9. Les athlètes ayant gagné une médaille dans une épreuve individuelle, mais ayant été disqualifiés
dans une autre (14 lignes).

In [ ]:
query="""
select distinct a.*
from Athlete a, RangIndividuel r1, RangIndividuel r2
where a.aid = r1.aid and a.aid = r2.aid and r1.epid <> r2.epid
and r1.rang <= 3 and r2.rang is null
"""

cursor=execute(connection,query,show=True)

Q10. Les athlètes ayant gagné (au moins) une médaille dans une épreuve individuelle (182 lignes).

In [ ]:
query="""
select a.*
from Athlete a
where a.aid in (select aid
from RangIndividuel r
where r.rang <= 3)
"""

cursor=execute(connection,query,show=True)

### NOT EXISTS / NOT IN

Q11.  Les noms de pays qui n’ont qu’une seule équipe (9 lignes).

In [ ]:
query="""
select distinct p.NomP
from Pays p, Equipe e
where p.codePays = e.codePays
and not exists (select * from Equipe e1
where e1.eqid <> e.eqid and e1.CodePays = e.codePays)
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct p.NomP
from Pays p, Equipe e
where p.codePays = e.codePays
and e.codePays NOT IN (select e1.codePays from Equipe e1, Equipe e2
where e1.eqid <> e2.eqid and e1.CodePays = e2.codePays)
"""

cursor=execute(connection,query,show=True)

Q12. Les athlètes ayant participé à exactement 2 épreuves individuelles (402 lignes).

In [ ]:
query="""
select distinct a.*
from Athlete a, RangIndividuel r1, RangIndividuel r2
where a.aid = r1. aid and r2.aid= a.aid and r1.epid <> r2.epid
and not exists (select * from RangIndividuel r3
where r3.aid = r1.aid and r3.epid <> r1.epid and r3.epid<>r2.epid)
"""

cursor=execute(connection,query,show=True)

## TME5

### Jointures

Q1. Les sports auxquels LESSER Erik a participé en supposant qu'il a participé aux épreuves
individuelles (2 lignes).

In [ ]:
query="""
select distinct s.*
from Sport s, Athlete a, RangIndividuel r, Epreuve e
where a.nomAth='LESSER' and a.prenomAth='Erik'
and a.aid = r.aid and r.epid = e.epid and e.sid = s.sid
"""

cursor=execute(connection,query,show=True)

Q2. Les athlètes ayant participé aux épreuves individuelles de (au moins) deux sports différents (2 lignes).

In [ ]:
query="""
select distinct a.nomath, a.prenomath
from Athlete a,RangIndividuel r1,Epreuve e1,RangIndividuel r2,Epreuve e2
where a.aid = r1.aid and a.aid = r2.aid
and e1.epid = r1.epid and a.aid = r1.aid
and e2.epid = r2.epid and a.aid = r2.aid and e1.sid <> e2.sid
"""

cursor=execute(connection,query,show=True)

### NOT EXISTS / NOT IN / IS NULL

Q3. Les dates de début et de fin des Jeux Olympique 2014.

In [ ]:
query="""
select distinct e1.dateDebut as Debut, e2.dateFin as Fin
from Epreuve e1, Epreuve e2
where not exists ( select * from Epreuve e3 where e3.dateDebut < e1.dateDebut)
and not exists ( select * from Epreuve e4 where e4.dateFin > e2.dateFin) 
"""

cursor=execute(connection,query,show=True)

Q4. Les pays qui ont des participants de moins de 18 ans ou de plus que 40 au 24-02-2016
(Attention : il faut éliminer les athlètes dont on ne connaît pas la date de naissance) (9 ou 7 lignes).

In [ ]:
query="""
select distinct p.codepays, p.nomp
from athlete a, pays p
where a.codepays=p.codepays and a.datenaissance is not null
and (2016-year(a.datenaissance) <= 18
or 2016-year(a.datenaissance) >= 40)
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct p.CODEPAYS, p.NOMP
from Pays p
where exists (select *
from Athlete a
where a.codePays=p.codePays and a.dateNaissance is not null
and (datediff(day,a.dateNaissance,parsedatetime('24-02-2016', 'dd-MM-yyyy')) < 18 * 365
or datediff(day,a.dateNaissance,parsedatetime('24-02-2016', 'dd-MM-yyyy')) > 40 * 365))
"""

cursor=execute(connection,query,show=True)

In [ ]:
query="""
select distinct p.CODEPAYS, p.NOMP
from ATHLETE a, PAYS p
where a.codePays=p.codePays and a.dateNaissance is not null
and (2016-year(a.DATENAISSANCE) < 18
or (2016-year(a.DATENAISSANCE) > 40)
or ((2016-year(a.DATENAISSANCE) = 18
and DAY_OF_YEAR(datenaissance) >
DAY_OF_YEAR(parsedatetime('24-02-2016', 'dd-MM-yyyy'))))
or ((2016-year(a.DATENAISSANCE) = 40
and DAY_OF_YEAR(datenaissance) <
DAY_OF_YEAR(parsedatetime('24-02-2016', 'dd-MM-yyyy')))))
"""

cursor=execute(connection,query,show=True)

Q5. Les équipes n'ayant pas d'athlète (la base de données ne contient pas l'information sur les participants) (13 lignes)

In [ ]:
query="""
select e.eqid, e.codePays
from Equipe e
where not exists (select *
from AthletesEquipe ae
where e.eqid = ae.eqid)
"""

cursor=execute(connection,query,show=True)

Q6. Les équipes qui ont exactement 2 athlètes. Retourner l'équipe en question avec ses deux seuls athlètes (114 lignes).


In [ ]:
query="""
select ae1.eqid, a1. nomath, a2.nomath
from athlete a1, athlete a2, athletesequipe ae1, athletesequipe ae2
where a1.aid<a2.aid and ae1.eqid=ae2.eqid and ae1.aid=a1.aid and ae2.aid=a2.aid
and not exists (select * from Athletesequipe ae3
where ae3.aid<>a1.aid and ae3.aid<>a2.aid and ae3.eqid=ae1.eqid)
"""

cursor=execute(connection,query,show=True)

Q7. Les sports qui n'ont pas d'épreuves de catégorie Mixte (12 lignes).

In [ ]:
query="""
select nomSp
from Sport s
where not exists (select *
from Epreuve e
where e.sid = s.sid and e.categorie='Mixte')
"""

cursor=execute(connection,query,show=True)

Q8. Les athlètes qui ont gagné une médaille d’or (au moins) mais pas de médaille d’argent ni de bronze (53 lignes)

In [ ]:
query="""
select distinct a.aid, a.nomath, a.prenomath
from athlete a, rangindividuel r
where a.aid= r.aid and r.rang=1
and a.aid not in (select r1.aid from Rangindividuel r1 where r1.rang=2 or r1.rang = 3)
"""

cursor=execute(connection,query,show=True)

Q9. Les athlète(s) qui ont fini dernier d’une épreuve individuelle. Indiquer leur nom, prenom, le nom
du sport et de l’épreuve et leur rang. Attention : il faut filtrer les athlètes disqualifiés (73 lignes).

In [ ]:
query="""
select a.aid, a.nomath, a.prenomath, r.rang, e.nomep, s.nomsp
from athlete a, rangindividuel r, epreuve e, sport s
where a.aid=r.aid and r.rang is not null
and e.epid=r.epid and s.sid=e.sid
and not exists (select * from rangindividuel r1
where r1.epid = r.epid and r.rang < r1.rang)
order by a.aid
"""

cursor=execute(connection,query,show=True)

# TD et TME 6

### TD 6 ANY /ALL / IN / EXISTS

Q1. Les athlètes ayant participé à (au moins) une épreuve individuelle et (au moins) une épreuve
par équipe. Exprimer la requête en utilisant ANY (au lieu de IN; voir requête 2.6 dans TD5).
Résultat : (372 lignes)

In [ ]:
query="""
select nomAth, prenomAth
from Athlete
where aid=ANY (select aid from RangIndividuel)
and aid=ANY (select aid from RangEquipe re,AthletesEquipe ae where re.eqid=ae.eqid)
"""
execute(connection,query,show=True)

2. a) Les pays ayant au moins un athlète (requête équivalente: les pays ayant participé aux JO).
Résultat : (88 lignes)

In [ ]:
query="""
select distinct p.nomP
from Pays p, Athlete a
where a.codePays = p.codePays
"""
execute(connection,query,show=True)

b) Les pays ayant exactement un seul athlète
Résultat : (18 lignes)

In [ ]:
query="""
select p.nomP
from Pays p, Athlete a1
where a1.codePays = p.codePays
and not exists (select *
from Athlete a2
where a2.aid <> a1.aid and a2.codePays = p.codePays)
"""
execute(connection,query,show=True)


3. a) Les athlètes qui n'ont jamais été disqualifiés aux épreuves individuelles.
Résultat : (2194 lignes)

In [ ]:
query="""
select nomAth, prenomAth
from Athlete
where aid NOT IN (select aid
from RangIndividuel
where rang is null)
"""
execute(connection,query,show=True)

b) Les pays qui n'ont pas eu d'athlète disqualifié aux épreuves individuelles.
Résultat : (143 lignes)

In [ ]:
query="""
select p.nomP
from Pays p
where not exists (select *
from Athlete a, RangIndividuel r
where a.codePays = p.codePays and a.aid = r.aid and r.rang is null)
"""
execute(connection,query,show=True)

c) Les pays qui ont participé aux JO et qui n'ont pas eu d'athlètes disqualifiés aux épreuves
individuelles.
Résultat : (25 lignes)

In [ ]:
query="""
select p.nomP
from Pays p
where exists (select * from Athlete a1 where a1.codePays = p.codePays)
and not exists (select *
from Athlete a, RangIndividuel r
where a.codePays = p.codePays and a.aid = r.aid and r.rang is null)
"""
execute(connection,query,show=True)

d) Les pays qui ont participé aux JO et qui n'ont pas eu des athlètes disqualifiés ni en
individuel ni par équipe
Résultat : (25 lignes)

In [ ]:
query="""
select p.nomP
from Pays p
where exists (select *
from Athlete a1
where a1.codePays = p.codePays)
and not exists (select *
from Athlete a, RangIndividuel r
where a.codePays = p.codePays and a.aid = r.aid and r.rang is null)
and not exists (select *
from Equipe e, RangEquipe re
where e.codePays = p.codePays and e.eqid = re.eqid and re.rang is null)
"""
execute(connection,query,show=True)

4. a) Les athlètes n'ayant pas gagné de médaille ni en individuel ni en équipe
Résultat : (1921 lignes)

In [ ]:
query="""
select nomAth, prenomAth
from Athlete a
where not exists (select *
from RangIndividuel r
where r.aid = a.aid and r.rang <= 3)
and not exists (select *
from AthletesEquipe e, RangEquipe r
where e.aid=a.aid and e.eqid = r.eqid and r.rang <= 3)
"""
execute(connection,query,show=True)

b) Les pays ayant participé aux JO et n'ayant pas gagné de médaille ni en individuel ni en
équipe.
Résultat : (63 lignes)

In [ ]:
query="""
select nomP
from Pays p
where exists (select * from Athlete a1 where a1.codePays = p.codePays)
and not exists (select *
from Athlete a2, RangIndividuel r
where a2.codePays = p.codePays and a2.aid = r.aid and r.rang <= 3)
and not exists (select *
from Equipe e, RangEquipe r
where e.codePays = p.codePays and e.eqid = r.eqid and r.rang <= 3)
"""
execute(connection,query,show=True)


5. L'(es) épreuve(s) avec la plus grande durée.
Résultat : Hockey sur glace, Hockey sur glace, Hommes, 16 jours

In [ ]:
query="""
select s.nomSp, e.nomEp, e.categorie, DATEDIFF(DAY, dateDebut, dateFin) + 1 as Duree
from Sport s, Epreuve e
where s.sid = e.sid and DATEDIFF(DAY, dateDebut, dateFin) >= all
(select DATEDIFF(DAY, dateDebut, dateFin) from Epreuve e2)
"""
execute(connection,query,show=True)

6. Les athlètes ayant gagné une médaille à toutes les épreuve individuelles auxquelles ils ont
participé. Résultat : (109 lignes)

In [ ]:
query="""
select distinct nomAth, prenomAth
From Athlete a, RangIndividuel r
Where a.aid = r.aid and NOT EXISTS (Select * From RangIndividuel i where i.aid=a.aid and
(i.rang> 3 OR i.rang IS NULL ))
"""
execute(connection,query,show=True)

In [ ]:
query="""
select distinct ri.aid
from RangIndividuel ri
where 3 >= all (select r.rang
from RangIndividuel r
where r.aid = ri.aid)
"""
execute(connection,query,show=True)

In [ ]:
query="""
select nomAth, prenomAth
from Athlete a
where 3 >= all (select r.rang
from RangIndividuel r
where r.aid = a.aid)
and a.aid in (select aid from RangIndividuel)
"""
execute(connection,query,show=True)

## TME6

### ANY /ALL / IN / EXISTS

Exprimer les requêtes suivantes en SQL :

1. Les sports dont toutes les épreuves ont duré un seul jour.
Résultat : Ski de fond, Ski alpin, Biathlon

In [ ]:
query="""
select nomSp
from Sport s
where not exists (select *
from Epreuve e
where e.sid = s.sid and e.dateFin <> e.dateDebut)
"""
execute(connection,query,show=True)

2. Les sports qui n'ont pas d'épreuves de categorie Mixte.
Résultat : (12 lignes)

In [ ]:
query="""
select nomSp
from Sport s
where not exists (select *
from Epreuve e
where e.sid = s.sid and e.categorie='Mixte')
"""
execute(connection,query,show=True)

3. Les équipes dont aucun athlète n'a gagné de médaille aux épreuves individuelles.
Attention : il y a des équipes sans athlètes.
Résultat : (265 lignes avec les équipes sans athlètes - 252 lignes sans les équipes sans
athlètes)

Avec les équipes sans athlètes :

In [ ]:
query="""
select e.eqid
from Equipe e
where not exists (select *
from AthletesEquipe ae, RangIndividuel ri
where e.eqid = ae.eqid and ri.aid=ae.aid and ri.rang <= 3)
"""
execute(connection,query,show=True)

Sans les équipes sans athlètes :

In [ ]:
query="""
select e.eqid
from AthletesEquipe e
where not exists (select *
from AthletesEquipe ae, RangIndividuel ri
where e.eqid = ae.eqid and ri.aid=ae.aid and ri.rang <= 3)
"""
execute(connection,query,show=True)

4. La nationalité de l'athlète le/la plus jeune. Attention : il y a des athlètes dont on ne connaît
pas la date de naissance.
Résultat : ('29/11/1998','JPN')

In [ ]:
query="""
select a1.dateNaissance, a1.codePays
from Athlete a1
where a1.dateNaissance is not null and not exists (select *
from Athlete a2
where a2.dateNaissance > a1.dateNaissance)
"""
execute(connection,query,show=True)

5. Le plus jeune athlète de chaque pays.
Résultat : (26 lignes)

In [ ]:
query="""
select a1.*
from Athlete a1, Pays p
where a1.codePays=p.codePays and a1.dateNaissance is not null
and not exists (select *
from Athlete a2
where a2.codePays=p.codePays
and a2.dateNaissance > a1.dateNaissance)
"""
execute(connection,query,show=True)

# TME7

On reprend le schéma schéma « Jeux Olympiques d'hiver 2014 ».

PAYS (CODEPAYS, NOMP)

SPORT (SID , NOM SP)

EPREUVE (EPID, SID*, NOMEP, CATEGORIE, DATEDEBUT, DATEFIN)

ATHLETE (AID, NOMATH, PRENOMATH, DATENAISSANCE, CODEPAYS*)

EQUIPE (EQID, CODE PAYS*)

ATHLETESEQUIPE (EQID*, AID*)

RANGINDIVIDUEL (EPID*, AID*, RANG)

RANGEQUIPE (EPID*, EQID*, RANG)

### Fonctions d’agrégation « COUNT, SUM, AVG, MIN, MAX »

1. Le nombre d’athlètes.
Résultat (1 ligne) : 2431

In [ ]:
query="""
select count(*) from Athlete
"""
execute(connection,query,show=True)

2. Le nombre d’athlètes ayant participé à au moins une épreuve en individuel.
Résultat (1 ligne) : 1558

In [ ]:
query="""
select count(distinct aid) from RangIndividuel
"""
execute(connection,query,show=True)

3. L'âge moyen des sportifs dont le code pays est 'FRA' (France) au 06/02/2014.
Résultat (1 ligne) : 26,8
Aide :
– utilisez round(valeur,nb) pour garder seulement nb décimales à valeur
– sous Oracle, utilisez: to_date('06/02/2014','dd/mm/YYYY')

In [ ]:
query="""
select round(avg(datediff(day,dateNaissance,'2014-02-06'))/365.25,1) as AgeMoy
from Athlete where codepays='FRA'
"""
execute(connection,query,show=True)

4. La durée moyenne, minimale et maximale des épreuves.
Résultat (1 ligne) : « Durée moyenne = 1,98 min = 1 max = 16 »
Aide : utilisez l’opérateur de concaténation ||
Attention : entre le 10/01/2014 et le 13/01/2014, il y a une durée de 4 jours (et non pas 3).

In [ ]:
query="""
select 'Durée moyenne = ' || round(avg(1.0+datediff(day, dateDebut, dateFin)),2)
|| ' min = ' || min(1+datediff(day, dateDebut, dateFin))
|| ' max = ' || max(1+datediff(day, dateDebut, dateFin))
from Epreuve
"""
execute(connection,query,show=True)

5. Le nombre moyen d'athlètes par pays, c'est-à-dire le nombre d'athlètes divisé par le nombre
de pays (ayant au moins un athlète). Résultat (1 ligne) : 27,625

In [ ]:
query="""
select Round(1.0*count(aid)/count(distinct codePays),3) as nbAthletesMoy
from Athlete
"""
execute(connection,query,show=True)

### Partitionnement « group by »

6. Pour chaque pays, le nom du pays et le nombre d’athlètes, ordonner par nombre d’athlètes
croissant.
Résultats (88 lignes) : (PAK,1) ; (HKG, 1) ; ... ; (USA, 196) ; (CAN,221)

In [ ]:
query="""
select codePays, count(*) as nbAthletes
from Athlete
group by codePays
order by count(*)
"""
execute(connection,query,show=True)

7. Le nombre moyen d'athlètes par pays (avec group by). Aide : compter le nombre
d’athlètes dans chaque pays (ayant au moins un athlète), puis faire la moyenne.
Résultat (1 ligne) : 27,625

In [ ]:
query="""
select avg(count) from (select count(*) as count from Athlete group by codePays) 
"""
execute(connection,query,show=True)

8. Pour chaque équipe, l’eqid de l'équipe et le nombre d'athlètes, ordonner par nombre
d’athlètes décroissant.
Résultats (296 lignes) : (164,25) ; (165,25) ; (166,25) ; ... ; (180,2) ; (181, 2) ; (182, 2)

In [ ]:
query="""
select eqid, count(*)
from AthletesEquipe
group by eqid
order by count(*) DESC
"""
execute(connection,query,show=True)

9. Pour chaque catégorie, la catégorie et le nombre d'épreuves.
Résultats (3 lignes) : (Femmes,43) ; (Mixte,6) ; (Hommes,49)

In [ ]:
query="""
select categorie, count(*)
from Epreuve
group by categorie
"""
execute(connection,query,show=True)

10. Pour chaque sport, le nom du sport et le nombre d'épreuves, ordonner par nombre d'épreuves
décroissant.

Résultats (15 lignes) : (Patinage de vitesse,12) ; (Ski de fond,12) ; ... ;(Hockey sur glace,2)

In [ ]:
query="""
select s.nomSp, count(*)
from Epreuve ep, Sport s
where ep.sid=s.sid
group by s.sid, s.nomSp
order by count(*) desc 
"""
execute(connection,query,show=True)

11. Pour chaque pays, le code du pays, le nombre de médailles en épreuve individuelle gagnées
et le nombre d'athlètes ayant gagnés au moins une médaille. Ordonner par nombre de
médailles décroissant. Aide : 2 tables seulement sont nécessaires.
Résultats (24 lignes) : (NOR, 24,19) ; (NED,22,15) ; ...

In [ ]:
query="""
select a.codePays, count(*), count(distinct a.aid)
from Athlete a, RangIndividuel ri
where a.aid=ri.aid and ri.rang <= 3
group by a.codePays
order by count(*) DESC
"""
execute(connection,query,show=True)

12. Pour chaque pays et sport, le code du pays, le sid du sport, le nombre de médailles en
épreuve individuelle gagnées, le nombre d'athlètes ayant gagnés au moins une médaille,
ordonner d'abord par code pays, puis par nombre de médailles décroissant.
Résultats (84 lignes) : (AUS,12,2,2); (AUS,15,1,1);(AUT,13,9,7);(AUT,15,2,2);...

In [ ]:
query="""
select a.codePays, e.sid, count(*), count(distinct a.aid)
from Athlete a, RangIndividuel ri, Epreuve e
where a.aid=ri.aid and ri.rang <= 3 and e.epid = ri.epid
group by a.codePays, e.sid
order by a.codePays, count(*) DESC
"""
execute(connection,query,show=True)

### Partitionnement avec « group by / having »

13. L’eqid de la ou des équipes qui sont composées :
a) d'exactement 10 athlètes. Résultat (1 ligne) : 226

In [ ]:
query="""
select eqid
from AthletesEquipe
group by eqid
having count(*) = 10
"""
execute(connection,query,show=True)

b) du plus d’athlètes pour ces JO.

Résultats (3 lignes) : 164 ; 165 ; 166

In [ ]:
query="""
select eqid from AthletesEquipe group by eqid
having count(*) = (select max(cnt) 
                    from  (select count(*) as cnt 
                           from AthletesEquipe
                           group by eqid
                )) 
"""
execute(connection,query,show=True)

In [ ]:
query="""
select eqid from AthletesEquipe group by eqid
having count(*) = (with R as (
                        select count(*) as cnt 
                           from AthletesEquipe
                           group by eqid )
                   select max(cnt) from  R) 
"""
execute(connection,query,show=True)

In [ ]:
query="""
select eqid from AthletesEquipe group by eqid
having count(*) >= ALL (select count(*) as cnt 
                    from AthletesEquipe
                   group by eqid) 
"""
execute(connection,query,show=True)

14. Le nombre d'épreuves en individuel où il y a eu au moins 100 participants.
Résultat (1 ligne ) : 2

In [ ]:
query="""
select count(c) from (select count(*) as c from Epreuve e, RangIndividuel ri where
e.epid=ri.epid group by e.epid having count(*) >= 100)
"""
execute(connection,query,show=True)

In [ ]:
query="""
with R as (
    select count(*) as c from Epreuve e, RangIndividuel ri 
    where e.epid=ri.epid 
    group by e.epid 
    having count(*) >= 100 
    )
select count(*) from R
"""
execute(connection,query,show=True)

15. Le nom des pays qui ont gagné au moins 20 médailles aux épreuves individuelles.
Résultats (3 lignes) : Pays-Bas ; États-Unis ; Norvège

In [ ]:
query="""
select p.nomP
from Athlete a, RangIndividuel ri, Pays p
where a.aid = ri.aid and ri.rang <= 3 and a.codePays=p.codePays
group by a.codePays, p.nomP
having count(*) >= 20
"""
execute(connection,query,show=True)

### Division en SQL

16. Le sid des sports qui ont des épreuves dans toutes les catégories existantes.
Résultats (3 lignes) : 1 ; 6 ; 7

Principe : pour chaque épreuve, on compte le nombre de catégories, puis on regarde si il est
égal au nombre total de catégories d’épreuves.

In [ ]:
query="""
select sid
from Epreuve
group by sid
having count(distinct categorie) = (select count(distinct categorie) from Epreuve)
"""
execute(connection,query,show=True)

In [ ]:
query="""
select sid
from Sport s
where not exists(select * from Epreuve e2
where not exists(select * from Epreuve e3
where e3.sid=s.sid and e3.categorie=e2.categorie))
"""
execute(connection,query,show=True)

17. Le nom des pays qui ont participé aux épreuves en individuel de tous les sports en

individuel. Résultats (3 lignes) : (Russie,12) ; (États-Unis,12) ; (Italie,12)

In [ ]:
query="""
select distinct p.nomP, count(distinct e.sid )
from Pays p, RangIndividuel ri, Athlete a, Epreuve e, Sport s
where p.codePays=a.codePays and a.aid=ri.aid and e.epid=ri.epid and s.sid=e.sid
group by p.codePays, p.nomP
having count(distinct e.sid ) = (select count(distinct e2.sid)
from Epreuve e2, RangIndividuel ri2
where e2.epid=ri2.epid)
"""
execute(connection,query,show=True)

## Requêtes avec paramètres

In [ ]:
cursor=connection.cursor()
cursor.execute("SELECT * FROM athlete WHERE aid=(%s)", '1')
print(cursor.fetchall())


# TD et TME 8   

## TD 8 Création de schémas (DDL)

On considère le schéma Entreprise décrit ci-dessous.

EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)

PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)

EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)

GRILLE_SAL (Profil, salaire)

La clé primaire de chaque relation est soulignée et les attributs des clés étrangères sont suivis d’un
astérisque. Cette base contient des informations sur des employés et sur les projets dans lesquels ils
sont impliqués. DateEnreg dans la table employe donne la date à laquelle l'employé a été enregistré
dans la BD. Ces employés sont embauchés dans un projet sur un profil donné et perçoivent un
salaire en fonction de ce profil. Le chef d'un employé dans la table Employé et le chef d'un projet
dans la table Projet sont des employés. En plus des contraintes de clé et contraintes référentielles
indiquées dans le schéma, on voudrait intégrer les contraintes suivantes :

* Domaines
    - Le numéro de sécurité sociale possède exactement 5 chiffres.
    - Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils peuvent en avoir moins).
    - La ville d’un employé (VillE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et sa longueur ne dépasse pas 9 caractères.
    - Le numéro d’un projet varie entre 5 et 7 chiffres.
    - Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.
    - Le budget est un entier sur 6 chiffres (sans virgule)
    - Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table Employé.

* Clés
    - Il n'y a pas deux employés avec le même nom et le même prénom.

* Not-null
    - Chaque projet doit avoir un responsable.
    - Dans la table Embauche, NumSS; NumProj et Profil représentent un employé, un projet et un profil existant. 


### Donner en SQL les instructions de création des contraintes de domaines indiquées dans
l'énoncé.

In [ ]:
connection = connect_H2(f"employes{port}","ba","ba",port)

In [ ]:
execute(connection,"drop all objects")

In [ ]:
query="""
/* Le numéro de sécurité sociale possède exactement 5 chiffres.*/
drop domain dnumss if exists;
create domain dnumss as numeric(5) check(length(value)=5)
"""
execute(connection,query)

In [ ]:
query="""
/* Le numéro de sécurité sociale possède exactement 5 chiffres.*/
drop domain dnumss if exists;
create domain dnumss as numeric(5) check(value >=10000 and value < 100000)
"""
execute(connection,query)

In [ ]:
query="""
/* Les attributs textuels (NomE, PrenomE, NomProj, Profil) ne dépassent pas 20 caractères (ils
peuvent en avoir moins).*/
drop domain dchaines if exists;
create domain dchaines as varchar(20)
"""
execute(connection,query)

In [ ]:
query="""
/* La ville d’un employé (VilleE) ou d’un projet (VilleP) se limite à 'Paris', 'Lyon' et 'Marseille' et
sa longueur ne dépasse pas 9 caractères.*/
drop domain dvilles if exists;
create domain dvilles as varchar(9) check(lower(value) in ('paris','lyon','marseille'))
"""
execute(connection,query)

In [ ]:
query="""
/*Le numéro d’un projet varie entre 5 et 7 chiffres.*/
drop domain dnumproj if exists;
create domain dnumproj as numeric(7) check (length(value)>=5);

/* Le salaire peut avoir deux chiffres après la virgule et ne dépasse pas 90 000.*/
drop domain dsal if exists;
create domain dsal as numeric(7,2) check (value <=90000);

/*Budget sur 6 chiffres*/
drop domain dbudget if exists;
create domain dbudget as numeric(6);

/*Aucun employé ne peut avoir plus de 70 ans au moment où il est enregistré dans la table
Employé.*/
drop domain ddatenaiss if exists;
create domain ddatenaiss as date check (datediff(year,value, current_date)<=70);
"""
execute(connection,query)


In [ ]:
query="""
SELECT domain_name,check_clause 
FROM information_schema.domain_constraints a, information_schema.check_constraints b
where a.constraint_name=b.constraint_name
"""
execute(connection,query)


In [ ]:
query="""
SELECT a.table_name,a.column_name,a.data_type FROM information_schema.columns a, information_schema.tables b
where a.table_name=b.table_name
and lower(a.table_name)='embauche'
"""
execute(connection,query)


### Donner en SQL les instructions de création du schéma de la base en leur associant les contraintes de clés, référentielles et de domaines indiquées dans l'énoncé.

In [ ]:
query="""
/*EMPLOYE (NumSS, NomE, PrenomE, NumChef*, VilleE, DateNaiss)*/
create table Employe(
    NumSS dnumss,
    NomE dchaines,
    PrenomE dchaines,
    NumChef dnumss,
    VilleE dvilles,
    DateNaiss ddatenaiss,
    /*contraintes de cle et referentielles*/
    constraint pk_emp primary key (NumSS),
    constraint fk_chef foreign key (numchef) references Employe,
    constraint key_emp unique(NomE, PrenomE) )
"""
execute(connection,query)


In [ ]:
query="""
/*PROJET(NumProj, NomProj, RespProj*, VilleP, Budget)*/
create table Projet(
NumProj dnumproj,
NomProj dchaines,
RespProj dnumss not null,
VilleP dvilles,
Budget dbudget,
/*contraintes de cle et referentielles*/
constraint pk_proj primary key(numproj),
constraint fk_resp foreign key (RespProj) references Employe
)
"""
execute(connection,query)




In [ ]:
query="""
/*GRILLE_SAL (Profil, salaire)*/
create table Grille_sal(
profil dchaines,
salaire dsal,
    
/*contraintes de cle et referentielles*/
constraint pk_sal primary key (profil) )
"""
execute(connection,query)


In [ ]:
query="""
/*EMBAUCHE (NumSS*, NumProj*, DateEmb, Profil*)*/
create table Embauche(
NumSS dnumss,
NumProj dnumproj,
DateEmb date,
Profil dchaines not null,
/*contraintes de cle et referentielles*/
constraint pk_emb primary key (NumSS, NumProj),
constraint fk_emb_emp foreign key (NumSS) references Employe,
constraint fk_emb_proj foreign key (NumProj) references Projet,
constraint fk_emb_sal foreign key (Profil) references Grille_sal
)
"""
execute(connection,query)


3. Un responsable de projet doit habiter la ville du projet dont il est responsable. Donner en
SQL l'instruction permettant d’exprimer cette contrainte en considérant que la table projet a été créée/ Utiliser la commande 

      alter table <table> add constraint <nom><expression> 

In [ ]:
query="""
alter table PROJET add constraint resprojet check (respproj in (select numss from employe
where villee = villep)) 
"""
execute(connection,query)


Commande utile: 
    

In [ ]:
query="""
alter table PROJET drop constraint resprojet
"""
execute(connection,query)


### Afficher le schéma

In [ ]:
show_table(connection, 'employe')

In [ ]:
show_schema(connection)

In [ ]:
query="""
SELECT t.table_name,constraint_name 
FROM information_schema.table_constraints t
"""
execute(connection,query,show=True)

In [ ]:
query="""
SELECT t.table_name,constraint_name,column_name,data_type 
FROM information_schema.table_constraints t, information_schema.columns c
where t.table_name=c.table_name
"""
execute(connection,query,show=True)

## TME 8 Modification de données (DML)

Dans ce qui suit, il vous est demandé d'effectuer des insertions de n-uplets dans des tables.

Syntaxe des insertions :

    insert into Table(attr1, attr2, ...) values ('val1', 'val2', ....)
    insert into Table values ('val1', 'val2', ....) 
    insert into Table <requête>
    

Syntaxe des suppressions :

    delete from table
    delete from table where <cond>

Syntaxe des mise à jour :

    update table set attr = valeur where <cond>
    update table set attr = <requete> where <cond>

In [ ]:
connection = connect_H2(f"employes{port}","ba","ba",port)

### Insérez dans chaque table au moins un n-uplet qui vérifie les contraintes d'intégrité. 

In [ ]:
query="""
insert into employe(NUMSS,NOME,PRENOME, VILLEE , DATENAISS ) values (12345, 'Smith', 'John', 'Paris', '1975-08-25' )
"""
execute(connection,query)

### Proposez des insertions qui violent les contraintes d'intégrité définies pour chaque table.

In [ ]:
query="""
insert into employe values(null,'LARS', 'Anna',null,'paris', parsedatetime('25-08-1975', 'dd-MM-yyyy' ));
"""
execute(connection,query)

Répondre aux questions suivantes :

1. Proposer une insertion dans la table Employé qui ne respecte pas la contrainte de clé primaire.

In [ ]:
query="""
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss) 
   values (21456, 'DUPONT', 'Marc', 'paris', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ;
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss) 
   values (21456, 'DUPOND', 'Pierre', 'paris', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ;
"""
execute(connection,query)

2. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de limite d'âge.

In [ ]:
query="""
insert into employe
 (NumSS, NomE, prenomE, villeE, dateNaiss)
values (45678, 'DUPONT', 'Marc', 'paris', parsedatetime('31-12-1944', 'dd-MM-yyyy' )) ;
"""
execute(connection,query)

3. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte de longueur de
l'attribut NumSS.

In [ ]:
query="""
insert into employe
 (NumSS, NomE, prenomE, villeE, dateNaiss)
values (456, 'DUPONT', 'Marc', 'paris', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ;
"""
execute(connection,query)

4. Proposer une insertion dans la table Employe qui ne respecte pas la contrainte sur les villes
possibles.

In [ ]:
query="""
insert into employe
 (NumSS, NomE, prenomE, villeE, dateNaiss)
values (45689, 'DUPONT', 'Marc', 'rennes', parsedatetime('12-01-1982', 'dd-MM-yyyy' )) ;
"""
execute(connection,query)

5. Insérer dans la table Employé deux employés avec le même nom et le même prénom.

In [ ]:
query="""
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
values (45678, 'LO', 'Dan', 'paris', parsedatetime('31-12-1984', 'dd-MM-yyyy')) ;
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
values (90123, 'LO', 'Dan', 'lyon', parsedatetime('01-02-1994', 'dd-MM-yyyy' )) ;
"""
execute(connection,query)

6. Proposer une insertion dans la table Grille_SAL qui ne respecte pas la contrainte sur le salaire
qui ne doit pas dépasser 90 000.

In [ ]:
query="""
insert into
 Grille_sal values ('dev', 99000) ;
"""
execute(connection,query)

7. Proposer une insertion dans la table Projet qui ne respecte pas la contrainte référentielle vers

Employe : insérer un responsable de projet qui n'est pas dans la table Employé
Insertion d'un projet avec pour responsable un employé avec numss = 97123 inexistant dans la
table Employe

Il est possible que la contrainte que le responsable soit dans la même ville est déclenchée. Vous pouvez enlever cette contrainte avec la commande :

    alter table drop constraint "nom de contrainte"
    
Il faudrait la recréer après.

In [ ]:
query="""
alter table PROJET drop constraint resprojet
"""
execute(connection,query)


In [ ]:
query="""
insert into Projet (numproj,nomproj,respproj)  values (123456,'infra', 97123) ;
"""
cursor=execute(connection,query)

In [ ]:
query="""
alter table PROJET add constraint resprojet check (respproj in (select numss from employe
where villee = villep)) 
"""
execute(connection,query)


8. Proposer une insertion dans la table Embauche qui ne respecte pas une des contraintes
référentielles : par exemple, associer un employé existant à un projet qui n'existe pas.

In [ ]:
query="""
insert into employe (NumSS, NomE, prenomE, villeE, dateNaiss)
       values (12345, 'DUPONT', 'Marc', 'paris', parsedatetime('12-01-1982', 'dd-MM-yyyy')) ;
insert into EMBAUCHE  values (12007, 45678, current_date,'Responsable');
"""
execute(connection,query)

# TD et TME 9 Création de schémas et modifications de données (DDL et DML)

### Villes et Pays

On veut créer un schéma relationnel pour stocker des informations sur des villes et des pays.

#### 1. Traduisez le schéma relationnel suivant en instructions SQL:

    Ville(nom, population, pays*)
    Pays(nom, capitale*)

où pays est une référence vers un pays dans la table Pays et capitale est une référence vers une ville dans la table Ville.

In [ ]:
connection = connect_H2(f"villes{port}","ba","ba",port)

In [ ]:
execute(connection,"drop all objects")

In [ ]:
query="""
create table Ville(
nom varchar(32) primary key,
population numeric(10),
pays varchar(32)
);
create table Pays(
nom varchar(32) primary key,
capitale varchar(32)
);
alter table Ville add constraint fk_pays foreign key (pays) references Pays(nom) ;
alter table Pays add constraint fk_ville foreign key (capitale) references Ville(nom) ;
"""
execute(connection,query)


#### Insérez la France avec sa capitale Paris (3 millions d'habitants) dans la base de données.

In [ ]:
query="""
insert into Ville values('Paris', 3000000, null);
insert into Pays values('France', 'Paris');
update Ville set pays = 'France' where nom = 'Paris';
"""
cursor=execute(connection,query)

autre solution :

In [ ]:
query="""
alter table Ville SET REFERENTIAL_INTEGRITY false ;
insert into Ville values('Paris', 3000000, 'France');
insert into Pays values('France', 'Paris');
alter table Ville SET REFERENTIAL_INTEGRITY true ;
"""
cursor=execute(connection,query)


In [ ]:
query="""
select * from Pays
"""
execute(connection,query,show=True)

In [ ]:
query="""
select * from Ville
"""
execute(connection,query,show=True)

#### Modifiez le schéma de telle manière que la suppression d'un pays déclenche automatiquement la suppression de toutes les villes du pays.
                                             

In [ ]:
query="""
alter table Ville drop constraint fk_pays;
alter table Ville add constraint fk_pays foreign key (pays) references Pays(nom) on delete
cascade ;

alter table Pays drop constraint fk_ville;
alter table Pays add
constraint fk_pays_capitale foreign key (capitale) references Ville(nom)
on update restrict ON DELETE CASCADE;
"""
execute(connection,query)



In [ ]:
query="""
delete from Pays;
select * from Pays
"""
execute(connection,query,show=True)

In [ ]:
query="""
select * from Ville
"""
execute(connection,query,show=True)

#### Effacez les deux tables Ville et Pays du schéma.

In [ ]:
query="""
alter table Ville drop constraint fk_pays if exists;
drop table Pays if exists;
drop table Ville;
"""
execute(connection,query)


### Arbres ordonnées

Voici une table Arbre qui permet de stocker des arbres ordonnés dans une base de données
relationnelle:

In [ ]:
connection = connect_H2(f"arbres{port}","ba","ba",port)

In [ ]:
query="""
drop table Arbre if exists;
create table Arbre(
    id numeric(10),
    par numeric(10),
    pos numeric(2),
    constraint pk primary key (id),
    constraint fk foreign key (par) references Arbre (id));
"""
execute(connection,query)


- id est l'identifiant du nœud,
- par est l'identifiant du parent,
- pos est la position parmi les enfants ;

#### Inserez l'arbre binaire (1(2(4,5),3(6,7(8,9))))) dans la base de données.

In [ ]:
query="""
delete from Arbre;
insert into Arbre values (1,NULL,0);
insert into Arbre values (2,1,0);
insert into Arbre values (3,1,1);
insert into Arbre values (4,2,0);
insert into Arbre values (5,2,1);
insert into Arbre values (6,3,0);
insert into Arbre values (7,3,1);
insert into Arbre values (8,7,0);
insert into Arbre values (9,7,1);
select * from Arbre
"""
execute(connection,query,show=True)


In [ ]:
query="""
delete from Arbre where par=7;
delete from Arbre where par=3;
delete from Arbre where id=3;
"""
execute(connection,query)


#### Comment faut il modifier le schéma pour empêcher qu'un noeud a deux enfants à la même position?

In [ ]:
query="""
alter table Arbre add constraint position_unique unique (par, pos);
"""
execute(connection,query)



In [ ]:
query="""
delete from Arbre where par=2;
delete from Arbre where id=2
"""
execute(connection,query)

#### Effacez le sous-arbre 3(6,7(8,9)) de la base de données.

In [ ]:
query="""
delete from Arbre where par=7;
delete from Arbre where par=3;
delete from Arbre where id=3
"""
execute(connection,query)


#### Est-ce que l'instruction suivante est possible sur l'arbre initial (avant l'effacement) ?

In [ ]:
query="""
delete from Arbre where id=3
"""
execute(connection,query)

In [ ]:
query="""
select * from Arbre
"""
execute(connection,query,show=True)

#### Comment peut-on modifier le schéma pour effacer le sous-arbre avec l'instruction précédente ?

In [ ]:
query="""
alter table Arbre drop constraint fk ;
alter table Arbre add constraint fk foreign key (par) references Arbre (id) on delete cascade;
"""
execute(connection,query)

#### Est-ce qu'il est possible de modifier l'identifiant d'un nœud ? 

#### Comment peut-on modifier le schéma pour permettre l'instruction précédente ?

In [ ]:
query="""
alter table Arbre drop constraint fk ;
alter table Arbre add constraint fk foreign key (par) references Arbre(id)
on update cascade on delete cascade;
update Arbre set id=15 where id=3;
select * from Arbre
"""
execute(connection,query,show=True)

#### Quel est le résultat des instructions suivantes (sur l'arbre initial):

In [ ]:
query="""
alter table Arbre drop constraint fk ;
alter table Arbre add constraint fk foreign key (par) references Arbre(id)
on delete set null;
delete from Arbre where id=3;
select * from Arbre
"""
execute(connection,query,show=True)

# Fermer la connexion

In [188]:
connection.commit() # implicit avec close
connection.close()

In [ ]:
connection